In [1]:
import string
import re
import multiprocessing

In [2]:
%%time
with open('ptwiki-latest-pages-articles.txt') as f:
    wiki_dump = [line for line in f if len(line.split()) > 10]
with open('../Indicadores/preprocessed_indicators.txt') as f:
    indicators = [w.rstrip().replace("_", " ") for w in f if "_" in w]

CPU times: user 24.5 s, sys: 1.18 s, total: 25.7 s
Wall time: 25.7 s


In [3]:
punctuation = string.punctuation \
        .replace('-', '') \
        .replace('_', '')
punctuation += "—«»"
punctuation = r"[{}]".format(punctuation)
re_remove_html = re.compile(r'<(\/|\\)?.+?>', re.UNICODE)
re_transform_emails = re.compile(r'[^\s]+@[^\s]+', re.UNICODE)
re_transform_url = re.compile(r'(http|https)://[^\s]+', re.UNICODE)
re_transform_numbers = re.compile(r'\b\d+[ªº°]*\b', re.UNICODE)
re_trim = re.compile(r' +', re.UNICODE)
re_tree_dots = re.compile(u'…', re.UNICODE)
def clean(text):
    text = text.lower()
    text = text.replace('\xa0', ' ')
    text = re_tree_dots.sub(' ', text)
    text = re_transform_url.sub('URL', text)
    text = re_transform_emails.sub('EMAIL', text)
    
    text = re.sub(punctuation, " ", text)
    text = re_transform_numbers.sub('0', text)
    text = re_trim.sub(' ', text)
#     text = text.split()
#     text = list(filter(lambda w: w not in stopwords, text))
#     text = ' '.join(text)
    for indicator in indicators:
            text = re.sub(fr'\b{indicator}\b', indicator.replace(" ", "_"), text)

    return text.strip()

In [4]:
%%time
pool = multiprocessing.Pool(processes=8)
wiki_dump = pool.map(clean, wiki_dump)

CPU times: user 18.7 s, sys: 8.52 s, total: 27.2 s
Wall time: 57min 42s


Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Process ForkPoolWorker-7:
Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Process ForkPoolWorker-8:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


In [5]:
with open("ptwiki-preprocessed.txt", 'wt') as f:
    for text in wiki_dump:
        f.write(text+'\n')
